In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = msm.convert([systems['nglview']['md_1u19.gro'], systems['nglview']['md_1u19.xtc']],
                      to_form='molsysmt.MolSys')
view = msm.convert(molsys, to_form='nglview.NGLWidget')

Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.


In [7]:
check_comparison = msm.compare(molsys, view, attribute_type='topological', rule='equal')

Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! T

In [6]:
msm.get(view, element='atom', chain_type=True)

Warning! The group name LRT is reserved in the Protein Data Bank for a small molecule, not for an amino acid.
Warning! The group name YPL is reserved in the Protein Data Bank for a small molecule, not for an amino acid.


['system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',
 'system',

In [ ]:
check_comparison

In [ ]:
check_comparison = msm.compare(molsys, view, comparison='info_no_form', rule='A_eq_B')
assert check_comparison